### Examples Of Pyspark ML

In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Missing').getOrCreate()

In [2]:
## Read The dataset
training = spark.read.csv('test1.csv',header=True,inferSchema=True)

In [3]:
training.show()



+------+---+----------+------+
|  Name|age|Experience|Salary|
+------+---+----------+------+
|   San| 28|         7| 30000|
|Prasad| 30|        10| 25000|
| Sunny| 28|         4| 20000|
|   Ram| 25|         5| 20000|
|Harsha| 22|         4| 15000|
|   Roy| 23|         2| 18000|
+------+---+----------+------+



In [4]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [5]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [7]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["age","Experience"],outputCol="Independent Features")

In [8]:
output=featureassembler.transform(training)

In [9]:
output.show()

+------+---+----------+------+--------------------+
|  Name|age|Experience|Salary|Independent Features|
+------+---+----------+------+--------------------+
|   San| 28|         7| 30000|          [28.0,7.0]|
|Prasad| 30|        10| 25000|         [30.0,10.0]|
| Sunny| 28|         4| 20000|          [28.0,4.0]|
|   Ram| 25|         5| 20000|          [25.0,5.0]|
|Harsha| 22|         4| 15000|          [22.0,4.0]|
|   Roy| 23|         2| 18000|          [23.0,2.0]|
+------+---+----------+------+--------------------+



In [10]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [11]:
finalized_data=output.select("Independent Features","Salary")

In [12]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|          [28.0,7.0]| 30000|
|         [30.0,10.0]| 25000|
|          [28.0,4.0]| 20000|
|          [25.0,5.0]| 20000|
|          [22.0,4.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [13]:
from pyspark.ml.regression import LinearRegression
##train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor=regressor.fit(train_data)

In [14]:
### Coefficients
regressor.coefficients

DenseVector([1000.0, 666.6667])

In [15]:
### Intercepts
regressor.intercept

-8666.666666665742

In [16]:
### Prediction
pred_results=regressor.evaluate(test_data)

In [17]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [23.0,2.0]| 18000|15666.666666666695|
|          [25.0,5.0]| 20000|19666.666666666715|
+--------------------+------+------------------+



In [20]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(1333.3333333332948, 2777777.7777776956)

In [22]:
pred_results.r2,pred_results.r2adj

(-1.7777777777776955, 3.7777777777776955)

In [23]:
pred_results.rootMeanSquaredError

1666.666666666642